In [3]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 30.0 MB/s eta 0:00:00


In [20]:
'''
IMPORT PART
'''

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer as Imputer
# from smdt import data_processing
# from smdt import molecular_descriptors

import numpy as np
import pandas as pd
import rdkit
import urllib.request
import json
from rdkit import Chem
from regression import fit_Ridge
from molecular_descriptors import getAllDescriptors as all_desc
from utils import *
from data_processing import *
#import descriptors

In [21]:
# copy the function of your choice from regression.py and the necessary imports for it -> it will perform hyperparameters tuning on selected regression model
# find and import the corresponding model from sklearn
# find and import the function that computes all molecular descriptors "from .molecular_descriptors.py import ..."

'''
DESCRIPTORS PART
'''

def Ridge(X_train, X_test, y_train, y_test):

    a = Imputer(missing_values=np.nan, strategy='median')
    b = StandardScaler()
    c = SelectKBest(score_func=mutual_info_regression)
    clf = RidgeCV(cv=10)
    model = Pipeline([('impute', a), ('scaling', b), ('anova', c), ('rf', clf)])

    # Grid Search CV
    parameters = {'anova__k': [5, 10, 20, 40]}

    grid = GridSearchCV(model, parameters)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    # Metrics
    metric = [grid.score(X_test, y_test),
               metrics.explained_variance_score(y_test, y_pred),
               metrics.mean_absolute_error(y_test, y_pred),
               metrics.mean_squared_error(y_test, y_pred),
               metrics.median_absolute_error(y_test, y_pred),
               metrics.r2_score(y_test, y_pred)]

    return grid, y_pred, metric

def desc_calc(data, mode='', log=None) -> pd.DataFrame:
    descriptors = all_desc(data, mode, log)
    return descriptors

def sar_model_evaluation(descriptors: pd.DataFrame):

    '''
    Function for model evaluation with functionCopyFromRegression().
    Saves best model.
    '''

    y = descriptors['Target']
    X = descriptors.drop(columns=['Target'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    model1, y_pred1, metrics1 = Ridge(X_train, X_test, y_train, y_test)
    return model1, y_pred1, metrics1

def sar_model_train(descriptors_train: pd.DataFrame, indices):

    '''
    Function for training the model with best paramaters.
    Don't forget to add here the input arguments required by the selected model.
    '''

    y_train = descriptors_train['Target']
    X_train = descriptors_train.drop(columns=['Target'])
    X_train = X_train[X_train.columns[indices]]

    a = Imputer(missing_values=np.nan, strategy='median')
    b = StandardScaler()
    clf = LassoCV()
    model = Pipeline([('impute', a), ('scaling', b), ('rf', clf)]) # without ANOVA now

    model.fit(X_train, y_train)
    return model

def sar_model_predict(model, descriptors_pred, indices):

    '''
    Function for casting predictions on unseen data
    '''

    X_pred = descriptors_pred
    X_pred = X_pred[X_pred.columns[indices]]
    return model.predict(X_pred)


'''
PUBCHEM PART
'''

def pubchem_parsing(url):

    '''
    Function for Pubchem request
    '''

    req = urllib.request.Request(url)
    res = urllib.request.urlopen(req).read()
    fin = json.loads(res.decode())
    return fin


def get_similar_cids(compound_smiles, threshold=95, maxentries=10):

    '''
    Function for finding similar CIDS in PubChem with fastsimilarity_2d
    '''

    pubchem_pug_rest_api_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/"
    pubchem_pug_rest_api_link+= "fastsimilarity_2d/smiles/%(smiles)s/cids/JSON?Threshold=%(threshold)s&MaxRecords=%(maxentries)s" % {
        "smiles": compound_smiles, "threshold": threshold, "maxentries": maxentries}
    similar_cids = pubchem_parsing(pubchem_pug_rest_api_link)['IdentifierList']['CID']
    return similar_cids


def get_xlogp(compound_cid):

    '''
    Function for parsing XLogP from the response
    '''

    pubchem_pug_rest_api_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/"
    pubchem_pug_rest_api_link += "compound/cid/%s/property/XLogP/JSON" % compound_cid

    try:
        xlogp = pubchem_parsing(pubchem_pug_rest_api_link)['PropertyTable']['Properties'][0]['XLogP']
        return xlogp
    except KeyError:
        return None


'''
MAIN PART
'''

if __name__ == "__main__":

    pd.set_option.use_inf_as_na = True

    # loading data
    # train_data = pd.read_csv('logp_100.csv')
    train_data = pd.read_csv('logp_full.csv')
    pred_data = pd.read_csv('logp_inputs.csv')
    cpds = [row for row in pred_data.loc[:, 'SMILES']]

    # calculating descriptors
    print("Calculating descriptors for training data...")
    train_descriptors = desc_calc(train_data, mode='train')
    print("Calculating descriptors for prediction data...")
    pred_descriptors = desc_calc(pred_data)
    # print(train_descriptors.head(), pred_descriptors.head())

    # finding best estimator
    print("Evaluating regression model parameters...")
    model = sar_model_evaluation(train_descriptors)
    print('Best parameters are:', model[0].best_params_) # {'anova__k': 40}
    cols = model[0].best_estimator_.named_steps['anova'].get_support(indices=True) # this are indices from ANOVA

    # add here other parameters from GridSearchCV best estimator (e.g. alpha for Ridge Regression)

    # train the best estimator and predict values
    print("Training the model with the best parameters...")
    final_model = sar_model_train(train_descriptors, cols)

    for cpd in cpds:
        cpd_descriptors = pred_descriptors[pred_descriptors['SMILES']==cpd]
        pred = sar_model_predict(final_model, cpd_descriptors, cols)
        print(f"Predicted LogP value for compound {cpd}:", pred)

        result = []

        print("Searching for similar compunds...")
        similarity = get_similar_cids(cpd) # related pubchem function

        print("Filtering logP...")
        for cid in similarity:
            xlogp = get_xlogp(cid) # related pubchem function
            if xlogp:
                if xlogp <= pred*1.1 and xlogp >=pred*0.9:
                    result.append((cid, xlogp))

        print(f"Request for compound {cpd} completed. I found the following CIDs in PubChem with XLogP in the range of {pred}+- 10%: {result}")

Calculating descriptors for training data...
Calculating Molecular Descriptors Completed.
Calculating descriptors for prediction data...
Calculating Molecular Descriptors Completed.
Evaluating regression model parameters...
Best parameters are: {'anova__k': 40}
Training the model with the best parameters...
Predicted LogP value for compound CC1=CC2=C(C=C1C)NC(=O)CC2C3=CC=CC=C3OC: [4.0178873]
Searching for similar compunds...
Filtering logP...
Request for compound CC1=CC2=C(C=C1C)NC(=O)CC2C3=CC=CC=C3OC completed. I found the following CIDs in PubChem with XLogP in the range of [4.0178873]+- 10%: [(2906028, 4), (17285549, 4.1)]
Predicted LogP value for compound COC1=CC=C2C(=C1)OC(CC2=O)(C(F)(F)F)O: [2.84387697]
Searching for similar compunds...
Filtering logP...
Request for compound COC1=CC=C2C(=C1)OC(CC2=O)(C(F)(F)F)O completed. I found the following CIDs in PubChem with XLogP in the range of [2.84387697]+- 10%: []


In [16]:
print(pred_descriptors)

       W        AW         J         Xu      GMTI   Pol    DZ       Ipc  \
0  844.0  4.019048  2.167149  20.028792  3.562412  36.0  44.5  4.843605   
1  553.0  3.614379  2.466824  17.337284  3.330617  32.0  44.5  3.901845   

    BertzCT      Thara  ...  ATSe8  ATSp1  ATSp2  ATSp3  ATSp4  ATSp5  ATSp6  \
0  2.849026  74.175794  ...   2.12  3.073  3.418  3.501  3.388  3.334  3.112   
1  2.696838  59.053571  ...   1.68  2.687  2.996  3.014  2.754  2.404  2.039   

   ATSp7  ATSp8  Target  
0  2.676  2.009    4.17  
1  1.667  0.670    2.79  

[2 rows x 759 columns]
